In [1]:
import numpy as np
import tensorflow as tf
train_path = np.load('/BrainSeg/data/patches_1024/train.npy')
val_path = np.load('/BrainSeg/data/patches_1024/val.npy')

In [2]:
# upsample function utilizing tf.keras.layers.Conv2D, size=4, strides=2, default set to be 2x resolution
# based on reference http://warmspringwinds.github.io/tensorflow/tf-slim/2016/11/22/upsampling-and-image-segmentation-with-tensorflow-and-tf-slim/
# upsample function, the size is determined by factor of images, strides is 2 * factor - factor % 2
def upsample(filters, size=4, strides=2, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=strides,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [3]:
def fcn_model(classes=3, drop_out_rate=0.2, bn=True):
    # use dropout and bacth normalization to prevent overfitting and help to do quick convergence
    # activation layer is added to incorporate non-linearity
    
    # input layer has variable length in width and height, tested on both 512, 1024
    input_imgs = tf.keras.layers.Input(shape=(None, None, 3))
    
    # All the kernel size, filter, stride are based on comparson paper, maxpooling layer is based on original FCN paper
    
    # First conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=1, padding='same')(input_imgs)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # Second conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=1, padding='same')(pool1)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # Third conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(pool2)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # Forth conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(pool3)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)  
    
    # Fifth conv layer + max pooling
    x = tf.keras.layers.Conv2D(filters=1024, kernel_size=11, strides=1, padding="same")(pool4)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    x = tf.keras.layers.Activation('relu')(x)
    
    pool5 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
    
    # build the fully connected layer using 1*1 convolutional layer
    x = tf.keras.layers.Conv2D(filters=512, kernel_size=1, strides=1, padding="same")(pool5)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    conv6 = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Conv2D(filters=classes, kernel_size=1, strides=1, padding="same")(conv6)
    x = tf.keras.layers.Dropout(drop_out_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x, training=bn)
    conv7 = tf.keras.layers.Activation('sigmoid')(x)

    # upsampling conv7 to 4x times and upsample pool4 to 2x times
    up_conv7 = upsample(filters=classes, size=8, strides=4)(conv7)
    up_pool4 = upsample(filters=64)(pool4)
    
    # Concatenate two resolutions
    fuse_1 = tf.keras.layers.Concatenate()([up_conv7, up_pool4])
    fuse_2 = tf.keras.layers.Concatenate()([fuse_1, pool3])
    
    prob = upsample(filters=classes, size=16, strides=8)(fuse_2)
    model = tf.keras.Model(inputs=input_imgs, outputs=prob)
    
    print(model.summary())
    print("FCN model building completes")
    
    return model

In [4]:
# Dataset class structure, combine with numpy array color normalization
from tensorflow.keras.utils import Sequence
from typing import Tuple
from nptyping import NDArray
from PIL import Image
class BrainSegSequence(Sequence):
    def __init__(self, image_paths: NDArray[str],
            mask_paths: NDArray[str], batch_size: int):
        self.image_paths = image_paths
        self.mask_paths  = mask_paths
        self.batch_size  = batch_size

    def __len__(self) -> int:
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, idx: int) -> Tuple[NDArray[np.uint8], NDArray[np.uint8]]:
        batch_x = self.image_paths[idx * self.batch_size : 
                (idx+1) * self.batch_size]
        batch_y = self.mask_paths[idx * self.batch_size : 
                (idx+1) * self.batch_size]
        return np.array([np.array(Image.open(p)) for p in batch_x])/255.0, \
                np.array([np.array(Image.open(p)) for p in batch_y])

In [5]:
# Construct the training and val dataset with batchsize 16
BATCH_SIZE = 16
train_dataset = BrainSegSequence(train_path[:,0], train_path[:,1], BATCH_SIZE)
val_dataset = BrainSegSequence(val_path[:,0], val_path[:,1], BATCH_SIZE)

In [6]:
def slide_filter(data_path, slide_index):
    filter_ls = []
    for ele in data_path:
        if ele[0].split('/')[-2] == slide_index:
            filter_ls.append(ele)
        elif ele[0].split('/')[-2] == slide_index+'17-24':
            filter_ls.append(ele)
    dataset = BrainSegSequence(np.array(filter_ls)[:,0], np.array(filter_ls)[:,1], BATCH_SIZE)
    return slide_index, len(filter_ls), dataset 

In [7]:
def slide_name_ls(address):
    # Input the address of the text file, read in the train file names, and val file names
    # return the length of train files, length of val files and train, val file lists
    f = open(address, "r")
    train_signal=False
    val_signal = False
    train_name_ls = [] 
    val_name_ls = []
    for line in f:
        if len(line.split('/')) > 1:
            if line.split('/')[4][:2] == 'NA' and train_signal:
                train_name_ls.append(line.split('/')[4][:12])
            elif line.split('/')[4][:2] == 'NA' and val_signal:
                val_name_ls.append(line.split('/')[4][:12])
        else:
            if line.split('/')[0] == '\tTrain: \n':
                train_signal = True
                val_signal = False
            elif line.split('/')[0] == '\tVal: \n':
                val_signal = True
                train_signal = False
    f.close()
    return len(train_name_ls), len(val_name_ls), train_name_ls, val_name_ls

In [8]:
_, _, train_file_ls, val_file_ls = slide_name_ls('/BrainSeg/data/patches_1024/dataset.txt')

In [17]:
def evaluate_slides(file_ls, path):
    # define and compile model again, set the batchnormalization training to be false to use accumulated statistics
    model = fcn_model(classes=3, bn=False)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[ tf.keras.metrics.SparseCategoricalAccuracy()])
    # return the latest weights in the folder
    latest = tf.train.latest_checkpoint('/BrainSeg/baseline_code/5_3_saved_weights/')
    # Load the previously saved weights
    model.load_weights(latest)
    for file in file_ls:
        slide_index, data_size, cur_dataset = slide_filter(path, file)
        # evaluate the model performance on this specific slide
        loss, acc = model.evaluate(cur_dataset)
        print("The {0} slide {1}: Accuracy: ".format('Validation', slide_index)+"{:5.2f}%".format(acc*100)+", Losses: {:5.4f}".format(loss))

In [223]:
evaluate_slides(val_file_ls, val_path)

Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_175 (Conv2D)             (None, None, None, 1 1216        input_26[0][0]                   
__________________________________________________________________________________________________
dropout_175 (Dropout)           (None, None, None, 1 0           conv2d_175[0][0]                 
__________________________________________________________________________________________________
batch_normalization_250 (BatchN (None, None, None, 1 64          dropout_175[0][0]                
___________________________________________________________________________________________

Non-trainable params: 3,570
__________________________________________________________________________________________________
None
FCN model building completes
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
207/207 [==============================] - 209s 1s/step - loss: 0.3356 - sparse_categorical_accuracy: 0.9083
The Validation slide NA4077-02_AB: Accuracy: 90.83%, Losses: 0.3356
183/183 [==============================]

IndexError: too many indices for array

In [16]:
evaluate_slides(train_file_ls, train_path)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, None, None, 1 1216        input_4[0][0]                    
__________________________________________________________________________________________________
dropout_21 (Dropout)            (None, None, None, 1 0           conv2d_21[0][0]                  
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, None, None, 1 64          dropout_21[0][0]                 
____________________________________________________________________________________________

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
156/156 [==============================] - 159s 1s/step - loss: 0.2859 - sparse_categorical_accuracy: 0.9272
The Validation slide NA3777-02_AB: Accuracy: 92.72%, Losses: 0.2859
152/152 [==============================] - 155s 1s/step - loss: 0.1180 - sparse_categorical_accuracy: 0.9619
The Validation slide NA4160-02_AB: Accuracy: 96.19%, Losses: 0.1180
135/135 [===============

In [25]:
evaluate_slides(val_file_ls[7:], val_path)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, None, None, 1 1216        input_4[0][0]                    
__________________________________________________________________________________________________
dropout_21 (Dropout)            (None, None, None, 1 0           conv2d_21[0][0]                  
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, None, None, 1 64          dropout_21[0][0]                 
____________________________________________________________________________________________

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
103/103 [==============================] - 96s 928ms/step - loss: 0.1048 - sparse_categorical_accuracy: 0.9640
The Validation slide NA4967-02_AB: Accuracy: 96.40%, Losses: 0.1048
115/115 [==============================] - 101s 879ms/step - loss: 0.3090 - sparse_categorical_accuracy: 0.8960
The Validation slide NA4972-02_AB: Accuracy: 89.60%, Losses: 0.3090
100/100 [==========